# 競馬予測

## 0. colabの環境を整える

### 0-1. git clone

In [ ]:
!git clone https://github.com/yuugo0724/keiba_prediction.git

### 0-2. 作業ブランチの作成

In [ ]:
# cloneしたディレクトリに移動
%cd ./keiba_prediction

# ブランチ名がmainであること
!git branch
# 作業ブランチの作成
!git branch [作業ブランチ名]
# 作業ブランチにチェックアウト
!git checkout [作業ブランチ名]
# 作業ブランチにチェックアウトできていることを確認
!git branch

### 0-2. ソースコードのディレクトリに移動

In [ ]:
%cd src/

### 0-3. pythonのライブラリをインストール

In [ ]:
pip install -r ../dockerfile/requirements.txt

## 1. モジュールやライブラリのインポート

### 1-1. インポート

In [3]:
"""
ライブラリ
"""
import os
import re
import subprocess
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#import lightgbm as lgb
from optuna.integration import lightgbm as lgb
import lightgbm as lgb_orig

"""
モジュール(定数)
"""
# ローカルパス
from modules.constants import LocalPaths
# データフレームの列名
#from modules.constants import ResultsCols
# レース名のマスター
from modules.constants import RaceInfo
# スクレイピングのパス
#from modules.constants import ScrapyPath

"""
モジュール(前処理)
"""
# 前処理
#from modules.preprocess import shaping
from modules.preprocess import _scrapy_data


### 1-2. ローカルパスの定義

In [4]:
# インスタンス化(path_list => pl)
lp = LocalPaths()
ri = RaceInfo()
# プロジェクトのベースディレクトリ
base_dir = lp.BASE_DIR
# scrapyのベースディレクトリ
scrapy_dir = lp.SCRAPY_DIR
# scrapy keibaプロジェクトのパス
scrapy_keiba_dir = lp.SCRAPY_KEIBA_DIR
# dataディレクトリ
data_dir = lp.DATA_DIR
# レース結果スクレイピング用url格納ディレクトリ
data_url_dir = lp.DATA_URL_DIR
# masterデータ格納ディレクトリ
data_master_dir = lp.DATA_MASTER_DIR
data_tmp_dir = lp.DATA_TMP_DIR
# gradesデータ格納ディレクトリ
data_grades_dir = lp.DATA_GRADES_DIR
# horse_gradesデータ格納ディレクトリ
data_horse_grades_dir = lp.DATA_HORSE_GRADES_DIR
# pedigreeデータ格納ディレクトリ
data_pedigree_dir = lp.DATA_PEDIGREE_DIR
# gradesのmaster
data_grades_master = lp.DATA_GRADES_MASTER
# horse_idのmaster
data_horse_id_master = lp.DATA_HORSE_ID_MASTER

# 競馬場ID
place_dict = ri.PLACE_DICT
# urlの正常性チェックプログラムのパス
#sy_unc_path = base_path + 'scrapy/url_normality_check.py'


## 2. 学習データ作成
2-1. スクレイピングを実施  
2-2. 前処理  
2-3. 学習データを説明変数と目的変数に分割  


### 2-1. スクレイピング
- スクレイピング対象のurlを取得  
- レース結果データを取得  
- 馬ごとの成績データを取得  
  ※学習データとしては、まだ利用していない  
- 血統データを取得  
  ※学習データとしては、まだ利用していない  

#### 2-1-1. 変数の定義

In [5]:
# レース結果スクレイピング対象のurl取得
proc_coll_url = lp.PROC_COLL_URL
log_coll_url = lp.LOG_COLL_URL
# レース結果スクレイピング
proc_coll_grades = lp.PROC_COLL_GRADES
log_coll_grades = lp.LOG_COLL_GRADES
# 馬ごとのレース結果スクレイピング
proc_coll_horse_grades = lp.PROC_COLL_HORSE_GRADES
log_coll_horse_grades = lp.LOG_COLL_HORSE_GRADES
# 馬ごとの血統データスクレイピング
proc_coll_pedigree = lp.PROC_COLL_PEDIGREE
log_coll_pedigree = lp.LOG_COLL_PEDIGREE


#### 2-1-2. スクレイピング対象のurl取得

In [ ]:
df_gen_date = pd.date_range(start="20120101",end="20221101", freq="MS")
df_date = df_gen_date.to_series().dt.strftime("%Y%m")
horse_id_list = df_date.values

os.chdir(scrapy_keiba_dir)
with open(log_coll_url, 'w') as f:
  for horse_id in horse_id_list:
    date_dir = os.path.join(data_url_dir,horse_id[0:4])
    os.makedirs(date_dir, exist_ok=True)
    scrapy_cmd = ["python3",proc_coll_url,horse_id,date_dir,data_url_dir]
    scrapy_proc = subprocess.Popen(scrapy_cmd, stdout=f, stderr=f)
    scrapy_proc.wait()
os.chdir(base_dir)

#### 2-1-3. レース結果のパスリスト作成

In [ ]:
url_file_list = []
for current_dir, sub_dirs, files_list in os.walk(data_url_dir):
  for file in files_list:
    url_file_list.append(os.path.join(current_dir,file))

#### 2-1-4. レース結果の取得

##### 2-1-4-1. 取得対象のレース期間を指定

In [ ]:
df_gen_date = pd.date_range(start="20200101",end="20210101", freq="MS")
df_date = df_gen_date.to_series().dt.strftime("%Y%m")
horse_id_list = df_date.values

target_url_file_list = []
for date in horse_id_list:
  date_match = '.*/' + date + '.csv'
  target_url_files = [url_file for url_file in url_file_list if re.match(date_match,url_file)]
  if target_url_files:
    target_url_file_list.extend(target_url_files)

##### 2-1-4-2. レース結果のスクレイピング

In [ ]:
os.chdir(scrapy_keiba_dir)
with open(log_coll_grades, 'w') as f:
  for url_file in target_url_file_list:
    file_name = url_file.split('/')[-1].split('.')[0]
    date_y = file_name[0:4]
    date_m = file_name[4:6]
    race_url_list = np.ravel(pd.read_csv(url_file,header=0).values.tolist())
    date_dir = os.path.join(data_grades_dir,date_y,date_m)
    os.makedirs(date_dir, exist_ok=True)
    for race_url in race_url_list:
      race_id = re.sub("\D","", race_url)
      scrapy_cmd = ["python3",proc_coll_grades,race_url,race_id,date_dir,data_grades_dir]
      scrapy_proc = subprocess.Popen(scrapy_cmd, stdout=f, stderr=f)
      scrapy_proc.wait()
os.chdir(base_dir)

# colabで取得する場合、時間がたつと切断されてしまうため、
# 取得後ブランチを作ってpushする
#!git add .
#!git commit -m "from colab"
#!git push


#### 2-1-5. 成績マスターの作成

In [6]:
race_list = []
for current_dir, sub_dirs, files_list in os.walk(data_grades_dir):
  for file in files_list:
    race_list.append(os.path.join(current_dir,file))
#print(race_list)
_scrapy_data.create_grades_master(race_list,data_grades_master)

#### 2-1-6. 成績マスターの読み込み

In [30]:
grades_master = pd.read_pickle(data_grades_master)

#### 2-1-7. 馬IDマスターの作成

In [ ]:
df_horse_id_master = grades_master['馬ID']
df_horse_id_master = df_horse_id_master.drop_duplicates()
df_horse_id_master = df_horse_id_master.reset_index(drop=True)
df_horse_id_master.to_pickle(data_horse_id_master)

#### 2-1-8. 馬のレース結果を取得
※現状学習データに含めるつもりはないので実施不要  
　今後、学習データに含める場合にコードを修正

In [ ]:
horse_id_list = ["2018104963","2018105074"]

os.chdir(scrapy_keiba_dir)
with open(log_coll_horse_grades, 'w') as f:
  for horse_id in horse_id_list:
    scrapy_cmd = ["python3",proc_coll_horse_grades,horse_id,data_horse_grades_dir]
    scrapy_proc = subprocess.Popen(scrapy_cmd, stdout=f, stderr=f)
    scrapy_proc.wait()
os.chdir(base_dir)

#### 2-1-9. 血統データを取得
※現状学習データに含めるつもりはないので実施不要  
　今後、学習データに含める場合にコードを修正

In [ ]:
horse_id_list = ["2018104963","2018105074"]

os.chdir(scrapy_keiba_dir)
with open(log_coll_pedigree, 'w') as f:
  for horse_id in horse_id_list:
    scrapy_cmd = ["python3",proc_coll_pedigree,horse_id,data_horse_grades_dir]
    scrapy_proc = subprocess.Popen(scrapy_cmd, stdout=f, stderr=f)
    scrapy_proc.wait()
os.chdir(base_dir)

### 2-2. 前処理

#### 2-2-1. 欠損値の削除

In [31]:
grades_master = grades_master.dropna(how='any')

#### 2-2-2. 体重増減を整数化(記号を削除)

In [32]:
grades_master['馬体重増減'] = grades_master['馬体重増減'].replace("+","").astype('int')

#### 2-2-3. 性齢の分割

In [33]:
sexual_age = grades_master['性齢']
sex = sexual_age.replace('[0-9]+',"", regex=True)
age = sexual_age.replace("\D","", regex=True)
grades_master['性'] = sex
grades_master['齢'] = age.astype(int)

#### 2-2-4. レース名の処理※要検討

In [34]:
place_id_list = []
race_id_list = grades_master['レースID']
for place_id in race_id_list:
  place_id_list.append(place_dict[place_id[4:6]])
grades_master['競馬場'] = place_id_list
#print(grades_master[['レースID','競馬場']])

In [35]:
grades_master['レース名'].drop_duplicates().to_csv('test.csv')

#### 競馬場・レース名を指定

In [36]:
# 競馬場指定
grades_master = grades_master[grades_master['競馬場']=='中山']

# レース名指定


#### 2-2-5. カテゴリ変数をダミー変数化

In [37]:
#grades_master = pd.get_dummies(grades_master,columns=['レース名'])
grades_master = pd.get_dummies(grades_master,columns=['回り'])
grades_master = pd.get_dummies(grades_master,columns=['天候'])
grades_master = pd.get_dummies(grades_master,columns=['タイプ'])
grades_master = pd.get_dummies(grades_master,columns=['馬場状態'])
grades_master = pd.get_dummies(grades_master,columns=['馬名'])
grades_master = pd.get_dummies(grades_master,columns=['騎手'])
grades_master = pd.get_dummies(grades_master,columns=['調教師'])
grades_master = pd.get_dummies(grades_master,columns=['性'])

#### 2-2-6. 不要な列を削除

In [38]:
grades_master = grades_master.drop('レースID', axis=1)
grades_master = grades_master.drop('レース名', axis=1)
grades_master = grades_master.drop('競馬場', axis=1)
grades_master = grades_master.drop('馬番', axis=1)
grades_master = grades_master.drop('性齢', axis=1)
grades_master = grades_master.drop('タイム', axis=1)
grades_master = grades_master.drop('単勝', axis=1)
grades_master = grades_master.drop('人気', axis=1)
grades_master = grades_master.drop('馬ID', axis=1)
grades_master = grades_master.drop('調教師ID', axis=1)

#### 2-2-7. データフレームの型をintに変換
変換対象列  
- 距離  
- 枠番  
- 斥量  
- 馬体重

スクレイピングのバグで距離列に空白が含まれていたのでそちらを削除  
※現時点では修正済み

In [ ]:
grades_master = grades_master[grades_master['距離'] != '']

In [39]:
grades_master['距離'] = grades_master['距離'].astype(int)
grades_master['枠番'] = grades_master['枠番'].astype(int)
grades_master['斥量'] = grades_master['斥量'].astype(float)
grades_master['馬体重'] = grades_master['馬体重'].astype(int)
grades_master['着順'] = pd.to_numeric(grades_master['着順'],errors='coerce')
grades_master = grades_master.dropna(how='any', axis=0)
grades_master['着順'] = grades_master['着順'].astype('int')

#### 2-2-7. 3着以内とそれ以外でデータを2分類化する

In [ ]:
grades_master.loc[grades_master['着順']<=3,['着順']] = 1
grades_master.loc[grades_master['着順']>3,['着順']] = 0

#### １着、２着、３着、それ以外で多分類化する

In [40]:
grades_master.loc[grades_master['着順']==1,['着順']] = 3
grades_master.loc[grades_master['着順']==2,['着順']] = 2
grades_master.loc[grades_master['着順']==3,['着順']] = 1

In [41]:
print(grades_master.dtypes)

距離            int64
着順            int64
枠番            int64
斥量          float64
馬体重           int64
             ...   
調教師_鹿戸雄一      uint8
調教師_黒岩陽一      uint8
性_セ           uint8
性_牝           uint8
性_牡           uint8
Length: 5163, dtype: object


### 2-3. 学習データを説明変数と目的変数に分割

In [42]:
#df_tran_data = df_tran_data.drop(['馬体重(増減)'], axis=1)

x = grades_master.drop(['着順'], axis=1)
y = grades_master['着順']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test)

(5918, 5162)
(2537, 5162)
(5918,)
(2537,)


#### 1-5-4. 学習データと検証データに分ける

### 1-6. 学習モデルの作成・学習

#### 1-6-1. ハイパーパラメータの設定

In [ ]:
params = {
  'objective': 'binary',
  'metric': 'auc',
}
best_params, histroy ={}, []
model = lgb.train(params,
                  lgb_train,
                  valid_sets=[lgb_train,lgb_eval],
                  num_boost_round=10,
                  early_stopping_rounds=10)
best_params_ = model.params

In [45]:
params = {
  'objective': 'multiclass',
  'num_class': 3,
  'metric': 'multi_logloss',
}
best_params, histroy ={}, []
model = lgb.train(params = params,
                  lgb_train = lgb_train,
                  valid_sets = [lgb_train,lgb_eval],
                  num_boost_round = 100,
                  early_stopping_rounds=5,
                  verbose_eval = 50
                  )
best_params_ = model.params

[I 2022-12-06 14:31:12,655] A new study created in memory with name: no-name-63151c10-65ac-449c-a7c4-ccfe40285159
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/home/keiba/.local/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
[W 2022-12-06 14:31:12,668] Trial 0 failed because of the following error: LightGBMError('Cannot set reference after freed raw data, set free_raw_data=False when construct Dataset to avoid this.')
Traceback (most recent call last):
  File "/home/keiba/.local/lib/python3.9/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/home/keiba/.local/lib/python3.9/site-packages/optuna/integration

LightGBMError: Cannot set reference after freed raw data, set free_raw_data=False when construct Dataset to avoid this.

#### 1-6-2. 学習モデル作成

In [44]:
model = lgb_orig.train(best_params_,
                        lgb_train,
                        valid_sets=lgb_eval,
                        num_boost_round=100,
                        early_stopping_rounds=10)

/home/keiba/.local/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Contains only one class
[LightGBM] [Info] Number of positive: 5918, number of negative: 0
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002065 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8641
[LightGBM] [Info] Number of data points in the train set: 5918, number of used features: 4238
[LightGBM] [Info] [binary:BoostFromScore]: pavg=1.000000 -> initscore=34.539576
[LightGBM] [Info] Start training from score 34.539576
[LightGBM] [Info] [binary:BoostFromScore]: pavg=1.000000 -> initscore=34.539576
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[1]	valid_0's auc: 1
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[2]	valid_0's auc: 1
[LightGBM] [Warning] Stopped training because there are no more leaves 

###

In [ ]:
pred = x_test.sample(n=1)
#print(pred)
y_pred = model.predict(pred)
print(y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred, labels=[1, 0]))